## Notebook imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [2]:
TRAINING_DATA_FILE = r'C:/Users/THE WIZRD/Desktop/SpamData/02_Training/train_data.txt'
TESTING_DATA_FILE = r'C:/Users/THE WIZRD/Desktop/SpamData/02_Training/test_data.txt'

VOCAB_SIZE = 2500

TOKEN_SPAM_PROB_FILE = r'C:/Users/THE WIZRD/Desktop/SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE= r'C:/Users/THE WIZRD/Desktop/SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = r'C:/Users/THE WIZRD/Desktop/SpamData/03_Testing/prob-all.txt'

TEST_FEATURE_MATRIX = r'C:/Users/THE WIZRD/Desktop/SpamData/03_Testing/test_features.txt'
TEST_TARGET_FILE = r'C:/Users/THE WIZRD/Desktop/SpamData/03_Testing/test_target.txt'



# Read and Load Features from .txt Files inot Numpy Array


In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TESTING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
print('Nr of rows in training files: ', sparse_train_data.shape[0])
print('Nr of rows in testing files: ', sparse_test_data.shape[0])

Nr of rows in training files:  257834
Nr of rows in testing files:  110522


In [7]:
print('Nr of unique emails in training files: ', np.unique(sparse_train_data[:,0]).size) 
print('Nr of unique emails in test files: ', np.unique(sparse_test_data[:,0]).size)

Nr of unique emails in training files:  4011
Nr of unique emails in test files:  1724


## Create an Empty DataFrame

In [8]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [9]:
len(column_names)

2502

In [10]:
index_names = np.unique(sparse_train_data[:,0])
index_names


array([   0,    1,    2, ..., 5789, 5791, 5793])

In [11]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)

In [12]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
full_train_data.fillna(0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a Full Matrix from a Sparse Matrix

In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    From a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the cocabulary. Total nuimber of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- poistion of occurrence of word in sparse matrix. Default: 4th column
    
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=index_names, columns=column_names)
    full_matrix.fillna(0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [15]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 16 s


In [16]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_data.tail()


,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5785,0,2,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5787,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5789,0,3,1,0,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5791,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,1,0,0,0
5793,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam 

In [18]:
prob_spam= len(full_train_data[full_train_data.CATEGORY == 1])/full_train_data.shape[0]
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.31089503864372975


## Total Number of Words / Token

In [19]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4011,)

In [21]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [22]:
total_wc = email_lengths.sum()
total_wc

426011

## Number of Tokens in Spam & Ham Emails

In [23]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape


(1247,)

In [24]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2764,)

In [25]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [26]:
spam_wc = spam_lengths.sum()
spam_wc

176056

In [27]:
ham_wc = ham_lengths.sum()
ham_wc

249955

## Summing the Tokens Occuring in Spam

In [28]:
full_train_features.shape

(4011, 2500)

In [29]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()
train_spam_tokens.shape

(1247, 2500)

In [30]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [31]:
summed_spam_tokens.tail()

2495    1
2496    1
2497    6
2498    1
2499    6
dtype: int64

In [32]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)
summed_ham_tokens.tail()

2495    26
2496    29
2497    21
2498     5
2499    18
dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam


In [33]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is NonSpam

In [34]:
prob_tokens_nonspam = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

0.9900972450535738

## P(Token) - Probability that Token Occurs

In [35]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [36]:
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)

## Prepare Test Data

In [37]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 6min 46s


In [38]:
full_test_features = full_test_data.loc[:, full_train_data.columns != 'CATEGORY']
full_test_target =  full_train_data.CATEGORY

In [39]:
np.savetxt(TEST_FEATURE_MATRIX, full_test_features)
np.savetxt(TEST_TARGET_FILE, full_test_target)